<a href="https://colab.research.google.com/github/emilny/schoolassignments/blob/main/profin6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Task 1

The latest stock price S and historic volatility (Annualized Standard Deviation of returns assuming 251 trading days a year) was gathered from *osebx2020.xlsx*

Furthermore, K is set to equal S since we are pricing an at-the-money option.

Since no time horizon T for the simulation was given (only the averaging period of "the last month" was specified in the task) a full year of 251 trading days was used for simulating paths. However, only the 21 last trading days were included in the average computation for the Asian option. 
In other words, each simulation has 251 steps (or days), and the final 21 days determine the option's payoff.

One clearly sees from experimentation that the variance in option value predicted here decreases as the number of simulations increases. In fact, since the option value is estimated as a mean, it's standard deviation is inversely proportional to the square root of ``` n_sims```.



In [6]:
import numpy as np

S = 946.3 # Last close
T = 1
K = S # At the money option
sigma = 0.181632015807428 # Approx historic volatility (annualized STDEV of returns) from excel

rf = 0.025
div = 0.04

n_steps = 251  # Number of trading days in a year (1255 rows from 27/11/20 to 27/11/15 divided by 5 is 251)
n_sims  = 50000
h = T/n_steps

u = (rf-div-0.5*sigma**2)*h

v = sigma*np.sqrt(h)

z = np.random.normal(0,1, size=(n_sims, n_steps))


In [7]:
S_mtc = np.zeros(shape=(n_sims, n_steps))
S_mtc[:,0] = S # All rows, on column 0 set to S

for j in range(1, n_steps):
  # This is where we simulate next step by multiplying previous step stock price
  S_mtc[:,j] = S_mtc[:,j-1] * np.exp(u+v*z[:,j])



In [8]:
# Now we need to calculate Asian option's value at the end of each simulation

avgs = np.average(S_mtc[:,-21:], axis=1)  # Arithmetic average of final 21 days

call_values = avgs - K  # Compute payoffs for Asian call option
call_values = np.where(call_values < 0, 0, call_values) # Replace negative payoffs with 0

put_values = K - avgs  # Compute payoffs for Asian put option
put_values = np.where(put_values < 0, 0, put_values) # Replace negative payoffs with 0

In [9]:
call_premium = np.average(call_values)*np.exp(-rf*T)
put_premium = np.average(put_values)*np.exp(-rf*T)


print(f"Call premium: {call_premium}\nPut premium: {put_premium}")

Call premium: 58.03634336763833
Put premium: 70.9067718988383


# Not part of the task, just some experimentation

Tesla options Monte Carlo simulation: 


Testing whether pricing in January of the September 500 puts was too high



In [10]:
S = 850 # Last close
T = 9/12  # Six and a half months until expiration
K = 500 # Strike price here, defauly is atm option
sigma = 0.577739711405303 # Approx. historic volatility (annualized STDEV of returns) from excel

rf = 0.001  # Based on US treasury bonds 6Mo and 1y yield
div = 0.00  # No dividend yield

n_steps = int(21*T*12)  # Number of trading days in a month times months to expiration
n_sims  = 500000  # Number of simulations to run
h = T/n_steps  # Time steps

u = (rf-div-0.5*sigma**2)*h
v = sigma*np.sqrt(h)
z = np.random.normal(0,1, (n_sims, n_steps))  # Assuming returns are lognormally distributed

# Begin simulation here
S_mtc = np.zeros(shape=(n_sims, n_steps))
S_mtc[:,0] = S # All rows, on column 0 set to S

for j in range(1, n_steps):
  # This is where we simulate next step by multiplying previous step stock price
  S_mtc[:,j] = S_mtc[:,j-1] * np.exp(u+v*z[:,j])

# Now we need to calculate European option's value at the end of each simulation
last_prices = S_mtc[:,-1]  # Final day price for all simulations

call_values = last_prices - K  # Compute payoffs for call option
call_values = np.where(call_values < 0, 0, call_values) # Replace negative payoffs with 0

put_values = K - last_prices  # Compute payoffs for put option
put_values = np.where(put_values < 0, 0, put_values) # Replace negative payoffs with 0

# Discount expected payoffs back to today
call_premium = np.average(call_values)*np.exp(-rf*T) 
put_premium = np.average(put_values)*np.exp(-rf*T)

print(f"Call premium: {call_premium}\nPut premium: {put_premium}")

Call premium: 373.17928207406123
Put premium: 23.486713922924086
